# Package

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
from sklearn.utils import shuffle
import pickle


d:\Portofolio Data science\Time Series\Travaux pratiques Bases\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importation des données

In [2]:
df_stationary_train = pd.read_csv("df_stationary_train.csv", index_col="date")
df_stationary_test = pd.read_csv("df_stationary_test.csv", index_col="date")

In [3]:
df_stationary_train.head()

,UNRATE,TB3MS,RPI,INDPRO,DPCERA3M086SBEA,S&P 500,BUSLOANS,CPIAUCSL,OILPRICEx,M2SL,USREC
date,,,,,,,,,,,
1960-01-01,-0.8,0.30,0.020977,0.091980,0.001204,0.017909,0.011578,-0.006156,0.0,0.001323,0
1960-02-01,-1.1,-0.19,0.014565,0.076964,0.006009,-0.025663,0.011905,-0.003767,0.0,0.002007,0
1960-03-01,-0.2,-1.18,0.006250,0.007961,0.021240,-0.070857,-0.008356,-0.005455,0.0,0.001324,0
1960-04-01,0.0,-1.12,0.006489,-0.025915,0.033752,-0.040442,-0.009098,0.005090,0.0,0.000634,1
1960-05-01,0.0,-0.67,0.007747,-0.018121,0.009040,-0.010090,-0.000359,0.003383,0.0,0.003977,1


In [4]:
df_stationary_train.index = pd.to_datetime(df_stationary_train.index)

# stationary

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# ==========================================================
# 🔧 Hypothèse : df_stationary_train contient la cible 'Y'
# ==========================================================

# --- Définir les features
features = [c for c in df_stationary_train.columns if c != "UNRATE"]

# === Paramètres ===
step_size = 12
winsor_level = 0.01
norm_var = True

# === Conteneurs ===
models = []
coefs = []
train_periods = []

In [6]:
range(step_size, len(df_stationary_train), step_size)

range(12, 360, 12)

In [7]:
a = range(1, 5 ,1)
print(a)

range(1, 5)


In [8]:
for t, end in enumerate(a):
    #print(t)
    print(end)

1
2
3
4


In [18]:
df_stationary_train.isna().sum()

UNRATE             0
TB3MS              0
RPI                0
INDPRO             0
DPCERA3M086SBEA    0
S&P 500            0
BUSLOANS           0
CPIAUCSL           0
OILPRICEx          0
M2SL               0
USREC              0
dtype: int64

In [14]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

try:
    from scipy.stats import pearsonr
    _has_scipy = True
except Exception:
    _has_scipy = False

# ====== Hyperparamètres ======
h = 12                  # horizon de prévision (de ta conf)
step_size = 12          # refit annuel
winsor_level = 0.01     # winsorisation 1% - 99%
norm_var = True         # normalisation z-score
target = "UNRATE"

# ====== Période TRAIN ======
train_start, train_end = "1960-01", "1989-12"
df = df_stationary_train.loc[train_start:train_end].copy()

# ====== Features : on utilise exactement celles que tu as déjà ======
cols_tx = list(features)

# S'assurer qu'on a des lignes complètes
df = df.dropna(subset=[target] + cols_tx).copy()

# ====== Containers ======
models, coefs, train_periods = [], [], []
yhat_list, ytrue_list, forecast_dates = [], [], []

# ====== Aides ======
def r2_origin_reg(y, yhat):
    if len(y) == 0: return np.nan
    denom = float(np.dot(yhat, yhat))
    if denom == 0: return np.nan
    b = float(np.dot(yhat, y)) / denom
    sse = float(np.sum((y - b * yhat) ** 2))
    sst = float(np.sum((y - y.mean()) ** 2))
    return 1 - (sse / sst) if sst > 0 else np.nan

def corr_pvalue(y, yhat):
    if len(y) < 3: return np.nan, np.nan
    if _has_scipy:
        r, p = pearsonr(y, yhat); return float(r), float(p)
    r = float(np.corrcoef(y, yhat)[0, 1])
    n = len(y)
    t_stat = r * np.sqrt((n - 2) / max(1e-12, 1 - r**2))
    from math import erf
    Phi = lambda x: 0.5 * (1 + erf(x / np.sqrt(2)))
    p = 2 * (1 - Phi(abs(t_stat)))
    return r, float(max(min(p, 1.0), 0.0))

# ====== Boucle rolling-origin (expanding) sur TRAIN ======
# Il faut au moins h+1 obs pour aligner (X_t, y_{t+h})
min_needed = h + 1
start_end = max(step_size, min_needed)

idx_all = df.index
for t, end in enumerate(range(start_end, len(df), step_size)):
    df_train_local = df.iloc[:end].copy()
    n = len(df_train_local)

    # Aligner (X_t, y_{t+h}) sans shift/dropna implicite
    X_train_raw = df_train_local[cols_tx].iloc[:n - h].copy()
    y_train = df_train_local[target].iloc[h:].copy()
    X_train_raw.index = y_train.index  # aligner les index temporels

    # Winsorisation (sur toutes les features — tes lags sont déjà dans les features)
    lower = X_train_raw.quantile(winsor_level)
    upper = X_train_raw.quantile(1 - winsor_level)
    X_train_w = X_train_raw.clip(lower=lower, upper=upper, axis=1)

    # Normalisation (z-score) avec stats de la fenêtre courante
    if norm_var:
        mean_ = X_train_w.mean()
        std_ = X_train_w.std().replace(0, 1)
        X_train = (X_train_w - mean_) / std_
    else:
        X_train = X_train_w

    # Fit (Ridge)
    model = Ridge(alpha=1.0, fit_intercept=True, random_state=0)
    model.fit(X_train, y_train)

    # Prévision à T+h avec X_T (dernière obs de la fenêtre)
    origin_date = df_train_local.index[-1]
    x_o = df_train_local.loc[origin_date, cols_tx].copy()
    x_o = x_o.clip(lower=lower, upper=upper)
    if norm_var:
        x_o = (x_o - mean_).divide(std_)

    yhat = float(model.predict(x_o.values.reshape(1, -1)))

    # Date cible T+h (évaluer seulement si encore dans TRAIN)
    end_pos = idx_all.get_loc(origin_date)
    target_pos = end_pos + h
    if target_pos < len(idx_all):
        y_true_date = idx_all[target_pos]
        if y_true_date <= pd.to_datetime(train_end):
            y_true = float(df.loc[y_true_date, target])
            yhat_list.append(yhat)
            ytrue_list.append(y_true)
            forecast_dates.append(y_true_date)

    # Sauvegardes
    models.append(model)
    coefs.append(pd.Series(model.coef_, index=cols_tx))
    train_periods.append(origin_date)

    print(f"[{t:02d}] Fin {origin_date.strftime('%Y-%m')} — modèle entraîné ({len(X_train)} obs). "
          f"Prévision pour {(y_true_date.strftime('%Y-%m') if target_pos < len(idx_all) else 'N/A')}: {yhat:.4f}")

# ====== Évaluation ROLLING sur TRAIN ======
y_true_arr = np.array(ytrue_list, dtype=float)
y_pred_arr = np.array(yhat_list, dtype=float)

if len(y_true_arr) == 0:
    print(f"\n⚠️ Aucune date cible (T+{h}) ne tombe dans le TRAIN {train_start}–{train_end}.")
else:
    r2 = r2_score(y_true_arr, y_pred_arr)
    r2_null = r2_origin_reg(y_true_arr, y_pred_arr)
    mae = mean_absolute_error(y_true_arr, y_pred_arr)             # métrique principale (MAE)
    rmse = np.sqrt(mean_squared_error(y_true_arr, y_pred_arr))    # compatible toutes versions de sklearn
    corr, pval = corr_pvalue(y_true_arr, y_pred_arr)
    amd = float(abs(np.mean(y_true_arr - y_pred_arr)))

    print("\n=== Évaluation TRAIN (rolling forecasts) — h={} ===".format(h))
    print(f"Période TRAIN    : {train_start} → {train_end}")
    print(f"Points évalués   : {len(y_true_arr)} "
          f"(de {pd.to_datetime(forecast_dates[0]).strftime('%Y-%m')} "
          f"à {pd.to_datetime(forecast_dates[-1]).strftime('%Y-%m')})")
    print("-----------------------------------------------")
    print(f"MAE              : {mae:.4f}")
    print(f"RMSE             : {rmse:.4f}")
    print(f"R²               : {r2:.4f}")
    print(f"R² (OLS b=0)     : {r2_null:.4f}")
    print(f"Corrélation      : {corr:.4f} (p={pval:.2e})")
    print(f"Abs Mean Dev.    : {amd:.4f}")
    print("-----------------------------------------------")
    print("✅ Backtest TRAIN terminé (prévisions à chaque fin de fenêtre).")

# ====== Sauvegarde ======
exp_results = {
    "models": models,
    "coefs": coefs,
    "train_periods": train_periods,
    "features": cols_tx,
    "forecast_dates": forecast_dates,
    "y_true": y_true_arr,
    "y_pred": y_pred_arr,
    "params": {
        "h": h,
        "step_size": step_size,
        "winsor_level": winsor_level,
        "norm_var": norm_var,
        "train_range": (train_start, train_end),
        "method": "Ridge",
        "lags_prebuilt": True  # ✅ on indique bien qu'ils étaient déjà dans les données
    },
    "metrics_train": {
        "mae": float(mae) if len(y_true_arr) else None,
        "rmse": float(rmse) if len(y_true_arr) else None,
        "r2": float(r2) if len(y_true_arr) else None,
        "r2_origin": float(r2_null) if len(y_true_arr) else None,
        "corr": float(corr) if len(y_true_arr) else None,
        "pval": float(pval) if len(y_true_arr) else None,
        "amd": float(amd) if len(y_true_arr) else None
    }
}


ValueError: Input X contains NaN.
Ridge does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# Evaluation globale

In [20]:
# === Évaluation globale (TRAIN, horizon h=24) ===
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_true = oos_train_h["UNRATE_true_h"].values
y_pred = oos_train_h["UNRATE_pred_h"].values

r2 = r2_score(y_true, y_pred)
r2_null = r2_score(y_true, np.zeros_like(y_true))  # baseline b=0
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
corr, pval = pearsonr(y_true, y_pred)
amd = np.mean(np.abs(y_true - np.mean(y_true)))

print("\n=== Évaluation TRAIN (h=24) ===")
print(f"R²              : {r2:.4f}")
print(f"R² (OLS b=0)    : {r2_null:.4f}")
print(f"MAE             : {mae:.4f}")
print(f"RMSE            : {rmse:.4f}")
print(f"Corrélation     : {corr:.4f} (p={pval:.2e})")
print(f"Abs Mean Dev.   : {amd:.4f}")

NameError: name 'oos_train_h' is not defined

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
import shap

# ==========================================================
# 📈 ÉVALUATION SUR LE TRAIN GLOBAL
# ==========================================================
features = exp_results["features"]
model_final = exp_results["models"][-1]  # dernier modèle
winsor_level = exp_results["params"]["winsor_level"]
norm_var = exp_results["params"]["norm_var"]

X_full = df_stationary_train[features].copy()
Y_full = df_stationary_train["UNRATE"].copy()

# Winsorisation + normalisation globales
lower_wins = X_full.quantile(winsor_level)
upper_wins = X_full.quantile(1 - winsor_level)
X_full = X_full.clip(lower=lower_wins, upper=upper_wins, axis=1)

if norm_var:
    mean_full = X_full.mean()
    std_full = X_full.std().replace(0, 1)
    X_full = (X_full - mean_full) / std_full

# === Prédictions et métriques
y_pred = model_final.predict(X_full)

r2 = r2_score(Y_full, y_pred)
r2_null = r2_score(Y_full, np.zeros_like(Y_full))
mae = mean_absolute_error(Y_full, y_pred)
rmse = np.sqrt(mean_squared_error(Y_full, y_pred))
corr, pval = pearsonr(Y_full, y_pred)
amd = np.mean(np.abs(Y_full - np.mean(Y_full)))

import joblib
import os

# Chemin de sauvegarde
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

# Fichier du modèle
model_path = os.path.join(model_dir, "model_final.pkl")

# Dictionnaire complet à sauvegarder
saved_model = {
    "model": model_final,
    "features": features,
    "winsor_level": winsor_level,
    "norm_var": norm_var,
    "mean_full": mean_full if norm_var else None,
    "std_full": std_full if norm_var else None,
}

# Sauvegarde avec joblib
joblib.dump(saved_model, model_path)

print(f"\n✅ Modèle sauvegardé avec succès : {model_path}")

print("\n=== Performance globale (TRAIN) ===")
print(f"R² (ensemble) : {r2:.4f}")
print(f"R² (OLS b=0)  : {r2_null:.4f}")
print(f"MAE            : {mae:.4f}")
print(f"RMSE           : {rmse:.4f}")
print(f"Corrélation    : {corr:.4f} (p = {pval:.2e})")
print(f"Abs Mean Dev.  : {amd:.4f}")


✅ Modèle sauvegardé avec succès : models\model_final.pkl

=== Performance globale (TRAIN) ===
R² (ensemble) : -0.1009
R² (OLS b=0)  : -0.0000
MAE            : 0.8792
RMSE           : 1.1751
Corrélation    : 0.5711 (p = 1.51e-32)
Abs Mean Dev.  : 0.8289


In [ ]:
# ==========================================================
# 🔍 IMPORTANCE PAR PERMUTATION
# ==========================================================
def permutation_importance(model, X, y, metric=mean_absolute_error, n_repeats=30):
    base_score = metric(y, model.predict(X))
    results = []
    for col in X.columns:
        scores = []
        for _ in range(n_repeats):
            X_shuffled = X.copy()
            X_shuffled[col] = np.random.permutation(X_shuffled[col])
            score = metric(y, model.predict(X_shuffled))
            scores.append(score)
        scores = np.array(scores)
        results.append({
            "variable": col,
            "perm_mae_ratio": np.mean(scores) / base_score,
            "perm_rmse_ratio": np.sqrt(mean_squared_error(y, model.predict(X_shuffled))) / rmse,
            "perm_deviance": np.mean(np.abs(scores - base_score))
        })
    return pd.DataFrame(results).sort_values("perm_mae_ratio", ascending=False)

perm_df = permutation_importance(model_final, X_full, Y_full)

print("\n=== Importance par permutation ===")
print(perm_df.head(10).to_string(index=False))


=== Importance par permutation ===
       variable  perm_mae_ratio  perm_rmse_ratio  perm_deviance
          USREC        1.291937         1.257301       0.188454
        S&P 500        1.127039         1.087991       0.082007
            RPI        1.095025         1.073389       0.061341
         INDPRO        1.013560         1.009293       0.008754
       BUSLOANS        1.011036         1.002059       0.007124
DPCERA3M086SBEA        1.010665         1.006124       0.007029
      OILPRICEx        1.010090         0.998821       0.006860
       CPIAUCSL        1.006380         1.002696       0.004358
          TB3MS        1.003201         1.002882       0.002403
           M2SL        0.998155         1.000433       0.001575


In [ ]:

# ==========================================================
# 💡 IMPORTANCE SHAPLEY
# ==========================================================
explainer = shap.Explainer(model_final, X_full)
shap_values = explainer(X_full)

shap_df = pd.DataFrame({
    "variable": X_full.columns,
    "shap_mean_abs": np.abs(shap_values.values).mean(axis=0),
})
shap_df["shap_share"] = shap_df["shap_mean_abs"] / shap_df["shap_mean_abs"].sum()
shap_df = shap_df.sort_values("shap_mean_abs", ascending=False)

print("\n=== Importance Shapley (shares) ===")
print(shap_df.head(10).to_string(index=False))


=== Importance Shapley (shares) ===
       variable  shap_mean_abs  shap_share
          USREC       0.340596    0.312587
        S&P 500       0.249643    0.229113
            RPI       0.185142    0.169917
         INDPRO       0.067299    0.061764
       BUSLOANS       0.052476    0.048161
DPCERA3M086SBEA       0.052252    0.047955
      OILPRICEx       0.046934    0.043074
       CPIAUCSL       0.043060    0.039519
           M2SL       0.027262    0.025020
          TB3MS       0.024941    0.022890


In [ ]:
# ==========================================================
# 🧩 COMPARATIF GLOBAL
# ==========================================================
merged_df = pd.merge(perm_df, shap_df, on="variable", how="outer")
print("\n=== Résumé comparatif (Permutation + SHAP) ===")
print(merged_df.head(10).to_string(index=False))


=== Résumé comparatif (Permutation + SHAP) ===
       variable  perm_mae_ratio  perm_rmse_ratio  perm_deviance  shap_mean_abs  shap_share
       BUSLOANS        1.011036         1.002059       0.007124       0.052476    0.048161
       CPIAUCSL        1.006380         1.002696       0.004358       0.043060    0.039519
DPCERA3M086SBEA        1.010665         1.006124       0.007029       0.052252    0.047955
         INDPRO        1.013560         1.009293       0.008754       0.067299    0.061764
           M2SL        0.998155         1.000433       0.001575       0.027262    0.025020
      OILPRICEx        1.010090         0.998821       0.006860       0.046934    0.043074
            RPI        1.095025         1.073389       0.061341       0.185142    0.169917
        S&P 500        1.127039         1.087991       0.082007       0.249643    0.229113
          TB3MS        1.003201         1.002882       0.002403       0.024941    0.022890
          USREC        1.291937         1.

# 📊 Métriques utilisées

## 1) Performance globale

**Coefficient de détermination (R²)**  
$$
R^2 = 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}
$$  
➡️ Part de la variance expliquée par le modèle (0 = pas mieux que la moyenne, 1 = parfait).

**Erreur absolue moyenne (MAE)**  
$$
MAE = \frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|
$$  
➡️ Écart absolu moyen entre valeurs réelles et prédites, robuste aux outliers.

**Erreur quadratique moyenne (RMSE)**  
$$
RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2}
$$  
➡️ Similaire au MAE mais pénalise davantage les grosses erreurs.

**Corrélation de Pearson**  
$$
\rho(y, \hat{y}) = \frac{\text{Cov}(y, \hat{y})}{\sigma_y \cdot \sigma_{\hat{y}}}
$$  
➡️ Mesure le degré de lien linéaire entre les prédictions et les observations.

**Abs Mean Deviance (AMD)**  
$$
AMD = \frac{1}{n}\sum_{i=1}^n |\hat{y}_i - \bar{\hat{y}}|
$$  
➡️ Écart moyen des prédictions par rapport à leur moyenne ; sert de référence pour la permutation prédiction-basée.

---

## 2) Importance par permutation
La relation entre Y et X dépend du temps. Quand on perturbe la série X (en la mélangeant), on casse ce lien, et si l’erreur augmente, cela montre que X est une variable clé pour expliquer Y.

**Ratio MAE**  
$$
PI^{MAE}_j = \frac{MAE^{(perm)}_j}{MAE^{(base)}}
$$  
➡️ Si > 1, la variable est utile pour réduire l’erreur absolue.

**Ratio RMSE**  
$$
PI^{RMSE}_j = \frac{RMSE^{(perm)}_j}{RMSE^{(base)}}
$$  
➡️ Si > 1, la variable aide à limiter les grosses erreurs.

**Déviance de prédiction**  
$$
PI^{dev}_j = \frac{1}{n}\sum_{i=1}^n \big|\hat{y}_i - \hat{y}^{(perm)}_{i,j}\big|
$$  
➡️ Mesure combien les prédictions changent quand on brouille une variable.

---

## 3) Importance Shapley

**Décomposition des prédictions**  
$$
\hat{y}_i = \phi_0 + \sum_{j=1}^p \phi_{ij}
$$  
➡️ Chaque prédiction est expliquée par une contribution \(\phi_{ij}\) par variable.

**Importance absolue moyenne**  
$$
\text{Mean}(|\phi_j|) = \frac{1}{n}\sum_{i=1}^n |\phi_{ij}|
$$  
➡️ Contribution moyenne (absolue) d’une variable sur toutes les prédictions.

**Shapley share**  
$$
\Gamma_j = \frac{\text{Mean}(|\phi_j|)}{\sum_{k=1}^p \text{Mean}(|\phi_k|)}
$$  
➡️ Part relative de la variable dans l’explication totale (somme des parts = 1).

## Interprétation des résultats 

### Performance globale 
- R² = 0.2266 → modèle OLS explique ~23 % de la variance du chômage US.
- MAE = 0.6774 → en moyenne, l’erreur absolue est de 0.68 points (dans l’unité de la variable cible).
- RMSE = 0.8750 → un peu plus élevé que le MAE, ce qui indique la présence de grosses erreurs ponctuelles.
- Corrélation = 0.4760 (p ≈ 10⁻³⁵) → lien positif et significatif entre prédictions et observations, mais seulement modéré. Ce qui peut expliquer la présence d'une relation 
- Abs Mean Deviance = 0.3370 → sert ici de référence pour l’importance prédiction-basée : les prédictions s’écartent en moyenne de 0.34 de leur propre moyenne.

Lecture : le modèle OLS capte une partie utile du signal, mais laisse beaucoup de variance inexpliquée. La corrélation faible illustre éventuellement la présence des relations non-linéaire, et non captées par OLS.

### 🔹 2. Importance par permutation
- INDPRO (Industrial Production) : la plus influente. Sa permutation augmente MAE de +19 % et RMSE de +20 %, avec une forte déviance de prédiction (0.41).
- TB3MS (Taux d’intérêt à 3 mois) : impact non négligeable, ratios ~1.02 et déviance ~0.10.
- BUSLOANS (Prêts commerciaux) : rôle similaire (MAE ratio 1.017, déviance ~0.09).
- S&P 500 : contribution modérée, ratios légèrement > 1.
- RPI, M2SL : influence plus faible mais perceptible.
- CPIAUCSL, OILPRICEx, DPCERA3M086SBEA : quasi neutres (ratios ≈ 1, déviance très faible).

Lecture : INDPRO domine largement la performance, les autres apportent des compléments mais plus modestes.

### 🔹 3. Importance Shapley (shares)
- INDPRO : ~52 % de l’explication totale des prédictions → cohérence parfaite avec la permutation.
- TB3MS (12 %) + BUSLOANS (12 %) : deux autres piliers importants.
- S&P 500 (9,7 %) : contribue de façon notable.
- M2SL (5 %), RPI (3 %), CPIAUCSL (3,5 %) : apports plus secondaires.
- OILPRICEx et DPCERA3M086SBEA (<2 %) : quasi négligeables dans ce modèle.

Lecture : INDPRO est la variable macroéconomique centrale, suivie par des indicateurs financiers (taux courts, prêts bancaires, marché actions).

# Test

In [ ]:
import joblib

saved_model = joblib.load("models/model_final.pkl")
model_final = saved_model["model"]
features = saved_model["features"]
winsor_level = saved_model["winsor_level"]
norm_var = saved_model["norm_var"]
mean_full = saved_model["mean_full"]
std_full = saved_model["std_full"]